In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm
from sklearn.metrics import balanced_accuracy_score
import os
import pandas as pd
from PIL import Image
import numpy as np

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import timm
from PIL import Image
import os
import pandas as pd
from sklearn.metrics import balanced_accuracy_score
from torchvision import transforms

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define constants
NUM_CLASSES = 10
IMAGE_SIZE = 224
BATCH_SIZE = 16
NUM_EPOCHS = 2

# Dataset class
class GastroImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.images = self._load_images()

    def _load_images(self):
        images = []
        for cls in self.classes:
            class_dir = os.path.join(self.root_dir, cls)
            for subdir in ['KID', 'KVASIR', 'SEE-AI']:
                subdir_path = os.path.join(class_dir, subdir)
                if os.path.exists(subdir_path):
                    for img_name in os.listdir(subdir_path):
                        if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                            img_path = os.path.join(subdir_path, img_name)
                            images.append((img_path, self.class_to_idx[cls]))
        return images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# Data Augmentation
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = GastroImageDataset(root_dir="/kaggle/input/misahubdataset/Dataset/training", transform=train_transform)
val_dataset = GastroImageDataset(root_dir="/kaggle/input/misahubdataset/Dataset/validation", transform=val_transform)
test_dataset = GastroImageDataset(root_dir="/kaggle/input/testdataset-misahub/Testing set/Images", transform=val_transform)

# Calculate class weights
class_counts = [3530, 332, 163, 149, 64, 52, 32, 26, 19, 17]
total_samples = sum(class_counts)
class_weights = torch.tensor([total_samples / (NUM_CLASSES * count) for count in class_counts]).to(device)

# DataLoader with WeightedRandomSampler
sample_weights = [class_weights[label] for _, label in train_dataset]
sampler = WeightedRandomSampler(sample_weights, len(sample_weights))

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# Define CoAtNet Model
class CoAtNet(nn.Module):
    def __init__(self, model_name='coatnet_3_224', num_classes=NUM_CLASSES):
        super(CoAtNet, self).__init__()
        self.model = timm.create_model(model_name, pretrained=False, num_classes=num_classes)

    def forward(self, x):
        return self.model(x)

# Initialize the model and move to device
model = CoAtNet().to(device)
model = nn.DataParallel(model)

# Focal Loss with class weights
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean', weight=None):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.weight = weight

    def forward(self, inputs, targets):
        ce_loss = nn.CrossEntropyLoss(weight=self.weight, reduction='none')(inputs, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# Optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Training loop
best_val_accuracy = 0
best_model_state = None

# Validation loop update to include both balanced accuracy and regular accuracy
for epoch in range(NUM_EPOCHS):
    # Training Phase
    model.train()
    train_loss, train_correct, train_total = 0, 0, 0
    train_targets, train_predictions = [], []

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()
        train_targets.extend(labels.cpu().numpy())
        train_predictions.extend(predicted.cpu().numpy())

    train_accuracy = 100. * train_correct / train_total
    train_balanced_accuracy = balanced_accuracy_score(train_targets, train_predictions)

    # Validation Phase
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    val_targets, val_predictions = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

            val_targets.extend(labels.cpu().numpy())
            val_predictions.extend(predicted.cpu().numpy())

    # Calculate metrics for validation
    val_accuracy = 100. * val_correct / val_total
    val_balanced_accuracy = balanced_accuracy_score(val_targets, val_predictions)

    # Print metrics for each epoch
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | "
          f"Train Acc: {train_accuracy:.2f}% | Train Balanced Acc: {train_balanced_accuracy:.2f} | "
          f"Val Acc: {val_accuracy:.2f}% | Val Balanced Acc: {val_balanced_accuracy:.2f}")

    # Save the best model state
    if val_balanced_accuracy > best_val_accuracy:
        best_val_accuracy = val_balanced_accuracy
        best_model_state = model.state_dict()

# Load the best model state
model.load_state_dict(best_model_state)

# Test loop
test_predictions, test_probabilities, test_image_paths = [], [], []
model.eval()
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)
        _, predicted = outputs.max(1)

        test_predictions.extend(predicted.cpu().numpy())
        test_probabilities.extend(probabilities.cpu().numpy())
        test_image_paths.extend([test_dataset.images[i][0] for i in range(len(images))])

# Save results to CSV
results_df = pd.DataFrame({
    'image_path': test_image_paths,
    'predicted_class': [train_dataset.classes[pred] for pred in test_predictions]
})
results_df.to_csv('test_predictions.csv', index=False)
print("Test predictions saved to 'test_predictions.csv'")

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
